# SQL with Spark

In [1]:
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 14:37:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_green = spark.read.parquet('../data/taxi_ingest_data/parts/green/*/*')

In [4]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- ehail_fee: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: float (nullable = true)



In [18]:
df_yellow = spark.read.parquet('../data/taxi_ingest_data/parts/yellow/*/*')

In [6]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- extra: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- improvement_surcharge: float (nullable = true)
 |-- total_amount: float (nullable = true)
 |-- congestion_surcharge: float (nullable = true)
 |-- airport_fee: float (nullable = true)



Same for the most part except for:

- green has `ehail_fee`
- green's datetime is `lpep_`; yellow is `tpep_`;
    - rename both dataset's datetime columns to just `pickup_datetime` and `dropoff_datetime`

The steps below roughly mimic the query used to build `fact_trips` table from week 4

In [19]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [20]:
col_yellow = set(df_yellow.columns)
common_cols = [col for col in df_green.columns if col in col_yellow]

df_green_sel = df_green \
    .select(common_cols) \
    .withColumn('service_type', F.lit('green'))

df_yellow_sel = df_yellow \
    .select(common_cols) \
    .withColumn('service_type', F.lit('yellow'))

In [21]:
# same as SQL's union all
df_trips_data = df_green_sel.unionAll(df_yellow_sel)
df_trips_data.groupby('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 2802931|
|      yellow|55553400|
+------------+--------+



Save the `df_trips_data` into a table; lifespan tied to Spark session

In [14]:
df_trips_data.registerTempTable('trips_data')

/opt/spark-3.3.1-bin-hadoop3/python/lib/pyspark.zip/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.


In [15]:
df_trips_data.createOrReplaceTempView('view_trips_data')

Query against it using regular SQL syntax

In [16]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    view_trips_data
GROUP BY
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 1734176|
|      yellow|30904308|
+------------+--------+



Recreating the monthly revenue by zone report via spark SQL:

In [24]:
df_monthly = spark.sql("""
select
    -- Reveneue grouping 
    PULocationID as revenue_zone,
    -- revert back to postgres style syntax, from bigquery
    date_trunc('month', pickup_datetime) as revenue_month, 
    service_type, 

    -- Revenue calculation
    sum(fare_amount) as revenue_monthly_fare,
    sum(extra) as revenue_monthly_extra,
    sum(mta_tax) as revenue_monthly_mta_tax,
    sum(tip_amount) as revenue_monthly_tip_amount,
    sum(tolls_amount) as revenue_monthly_tolls_amount,
    -- sum(ehail_fee) as revenue_monthly_ehail_fee,
    sum(improvement_surcharge) as revenue_monthly_improvement_surcharge,
    sum(total_amount) as revenue_monthly_total_amount,
    sum(congestion_surcharge) as revenue_monthly_congestion_surcharge,

    -- Additional calculations
    count(1) as total_monthly_trips,
    avg(passenger_count) as avg_monthly_passenger_count,
    avg(trip_distance) as avg_monthly_trip_distance

    from view_trips_data
    group by revenue_zone, revenue_month, service_type
    """)

Writing the result back to our "Data Lake"

In [25]:
df_monthly.coalesce(4).write.parquet('../data/reports/revenue/')

## `Group By` in Spark

In [ ]:
from pathlib import Path

from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 14:37:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [19]:
parts_dir = Path('../data/taxi_ingest_data/parts/')

In [ ]:
df_green = spark.read.parquet(str(parts_dir / 'green/*/*'))

In [4]:
# enable SQL query via spark.sql
df_green.createOrReplaceTempView('green')

In [14]:
df_green_rev = spark.sql(
    sqlQuery="""
    SELECT
        date_trunc('hour', lpep_pickup_datetime) AS hour,
        PULocationID AS revenue_zone,
        CAST(SUM(total_amount) AS DECIMAL(6,2)) AS amount,
        COUNT(1) AS num_records
    FROM
        green
    WHERE
        CAST(lpep_pickup_datetime as DATE) BETWEEN "2020-01-01" AND "2021-12-31"
    GROUP BY
        hour, revenue_zone
    ORDER BY
        hour, revenue_zone
    """
)

In [15]:
df_green_rev.show()

+-------------------+------------+-------+-----------+
|               hour|revenue_zone| amount|num_records|
+-------------------+------------+-------+-----------+
|2020-01-01 00:00:00|           7| 769.73|         45|
|2020-01-01 00:00:00|          17| 195.03|          9|
|2020-01-01 00:00:00|          18|   7.80|          1|
|2020-01-01 00:00:00|          22|  15.80|          1|
|2020-01-01 00:00:00|          24|  87.60|          3|
|2020-01-01 00:00:00|          25| 531.00|         26|
|2020-01-01 00:00:00|          29|  61.30|          1|
|2020-01-01 00:00:00|          32|  68.95|          2|
|2020-01-01 00:00:00|          33| 317.27|         11|
|2020-01-01 00:00:00|          35| 129.96|          5|
|2020-01-01 00:00:00|          36| 295.34|         11|
|2020-01-01 00:00:00|          37| 175.67|          6|
|2020-01-01 00:00:00|          38|  98.79|          2|
|2020-01-01 00:00:00|          40| 168.98|          8|
|2020-01-01 00:00:00|          41|1363.96|         84|
|2020-01-0

In [17]:
rev_report_dir = Path("../data/reports/revenue/")

In [18]:
df_green_rev \
    .repartition(20) \
    .write \
    .parquet(str(rev_report_dir / 'green'))

In [20]:
df_yellow = spark.read.parquet(str(parts_dir / 'yellow/*/*'))

In [21]:
df_yellow.createOrReplaceTempView('yellow')

In [24]:
df_yellow_rev = spark.sql(
    sqlQuery="""
    SELECT
        date_trunc('hour', tpep_pickup_datetime) AS hour,
        PULocationID AS revenue_zone,
        CAST(SUM(total_amount) AS DECIMAL(6,2)) AS amount,
        COUNT(1) AS num_records
    FROM
        yellow
    WHERE
        CAST(tpep_pickup_datetime as DATE) BETWEEN "2020-01-01" AND "2021-12-31"
    GROUP BY
        hour, revenue_zone
    ORDER BY
        hour, revenue_zone
    """
)

In [25]:
df_yellow_rev.show()

+-------------------+------------+-------+-----------+
|               hour|revenue_zone| amount|num_records|
+-------------------+------------+-------+-----------+
|2020-01-01 00:00:00|           3|  25.00|          1|
|2020-01-01 00:00:00|           4|1004.30|         57|
|2020-01-01 00:00:00|           7| 455.17|         38|
|2020-01-01 00:00:00|          10|  42.41|          2|
|2020-01-01 00:00:00|          12| 107.00|          6|
|2020-01-01 00:00:00|          13|1214.80|         56|
|2020-01-01 00:00:00|          14|   8.80|          1|
|2020-01-01 00:00:00|          15|  34.09|          1|
|2020-01-01 00:00:00|          17| 220.21|          8|
|2020-01-01 00:00:00|          18|   5.80|          1|
|2020-01-01 00:00:00|          24| 754.95|         45|
|2020-01-01 00:00:00|          25| 324.35|         16|
|2020-01-01 00:00:00|          32|  18.00|          1|
|2020-01-01 00:00:00|          33| 255.56|          8|
|2020-01-01 00:00:00|          34|  19.30|          1|
|2020-01-0

In [26]:
df_yellow_rev \
    .repartition(20) \
    .write \
    .parquet(str(rev_report_dir / 'yellow'))

## `JOIN` in Spark

Let's join the two tables above, `df_green_rev` and `df_yellow_rev` on the keys `revenue_zone` and `hour`

In [29]:
df_rev = \
    df_green_rev \
        .withColumnRenamed('amount', 'green_amount') \
        .withColumnRenamed('num_records', 'green_num_records') \
    .join(
        df_yellow_rev \
            .withColumnRenamed('amount', 'yellow_amount') \
            .withColumnRenamed('num_records', 'yellow_num_records'),
        on=['hour', 'revenue_zone'],
        how='outer')

In [30]:
df_rev.show()

+-------------------+------------+------------+-----------------+-------------+------------------+
|               hour|revenue_zone|green_amount|green_num_records|yellow_amount|yellow_num_records|
+-------------------+------------+------------+-----------------+-------------+------------------+
|2020-01-01 00:00:00|           3|        null|             null|        25.00|                 1|
|2020-01-01 00:00:00|           4|        null|             null|      1004.30|                57|
|2020-01-01 00:00:00|           7|      769.73|               45|       455.17|                38|
|2020-01-01 00:00:00|          12|        null|             null|       107.00|                 6|
|2020-01-01 00:00:00|          37|      175.67|                6|       161.61|                 7|
|2020-01-01 00:00:00|          40|      168.98|                8|        89.97|                 5|
|2020-01-01 00:00:00|          45|        null|             null|       732.48|                42|
|2020-01-0

The `null`s come from the fact we did an `outer join`; the num_records and amount that are null came from the other table, where there were no trips for that `hour` and `revenue_zone`

In [31]:
df_rev.repartition(20).write.parquet(str(rev_report_dir / 'total'))

In [33]:
df_zones = spark \
    .read \
    .option('header', 'true') \
    .csv('../data/taxi_ingest_data/taxi_zone_lookup.csv')
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [34]:
df_result = df_rev.join(df_zones, df_rev.revenue_zone == df_zones.LocationID)
df_result.show()

+-------------------+------------+------------+-----------------+-------------+------------------+----------+---------+--------------------+------------+
|               hour|revenue_zone|green_amount|green_num_records|yellow_amount|yellow_num_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+------------+------------+-----------------+-------------+------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|           3|        null|             null|        25.00|                 1|         3|    Bronx|Allerton/Pelham G...|   Boro Zone|
|2020-01-01 00:00:00|           4|        null|             null|      1004.30|                57|         4|Manhattan|       Alphabet City| Yellow Zone|
|2020-01-01 00:00:00|           7|      769.73|               45|       455.17|                38|         7|   Queens|             Astoria|   Boro Zone|
|2020-01-01 00:00:00|          12|        null|             null|       107.

In [35]:
df_result = df_result.drop('LocationID', 'revenue_zone')
df_result.write.parquet(str(rev_report_dir / 'total_with_zones'))